## Imports

In [16]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.linear_model import *
from sklearn.ensemble import *
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import *

from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime
import math

from joblib import dump, load

lookup_df = pd.read_csv("zone_lookup.csv")

DATEPARSE = "%Y-%m-%d %H"

In [15]:
def save_model(model, poly, min_max):
    file_name = datetime.now().strftime(DATEPARSE)
    dump(model, "models/" + file_name + "_model.joblib")
    dump(poly, "models/" + file_name + "_poly.joblib")
    dump(min_max, "models/" + file_name + "_min_max.joblib")

# Train

In [17]:
#dfGreen = pd.read_csv("taxi_data/green_tripdata_2019_concat.csv", chunksize=50000)
dfYellow = pd.read_csv("taxi_data/yellow_tripdata_2019_concat.csv", chunksize=10000)

model = SGDRegressor()

poly = PolynomialFeatures(degree=12, include_bias=False)
min_max = MinMaxScaler()

counter = 0
best_accuracy = 0

for chunk in dfYellow:

    X_green = chunk[["PU_start", "PUlat", "PUlong","DOlat", "DOlong"]] # input cols
    y_green = chunk[["travel_time"]] # output cols
    
    scaled = poly.fit_transform(min_max.fit_transform(X_green.values))

    X_train, X_test, y_train, y_test = train_test_split(scaled, y_green.values.ravel(), test_size = 0.25)

    model.partial_fit(X_train, y_train)

    y_pred = model.predict(X_test)
    
    score = r2_score(y_test, y_pred)
    
    print(str(counter) + ": Mean squared error: " + str(mean_squared_error(y_test, y_pred)) + " Accuracy: " + str(score))
    if score > best_accuracy:
        print("New best accuracy")
        save_model(model, poly, min_max)
        best_accuracy = score
    counter += 1

0: Mean squared error: 92.57406133821512 Accuracy: 0.003460095925976736
New best accuracy
1: Mean squared error: 88.57688403234954 Accuracy: 0.004814531606774697
New best accuracy
2: Mean squared error: 94.50435319292427 Accuracy: 0.0013553999610413925
3: Mean squared error: 80.67624269625807 Accuracy: 0.0174783765385641
New best accuracy
4: Mean squared error: 70.65385679564898 Accuracy: 0.0012311986297424138
5: Mean squared error: 81.57863892454749 Accuracy: -0.0655656147755912
6: Mean squared error: 78.38214931602552 Accuracy: 0.08648965622088
New best accuracy
7: Mean squared error: 55.21797078875144 Accuracy: 0.08079587422190593
8: Mean squared error: 53.58047056101465 Accuracy: 0.07360135073133611
9: Mean squared error: 71.6689257515843 Accuracy: 0.11328025858422608
New best accuracy
10: Mean squared error: 61.77138176011732 Accuracy: 0.18251452088906994
New best accuracy
11: Mean squared error: 77.25344726876142 Accuracy: 0.15123093102656404
12: Mean squared error: 81.0363930535

KeyboardInterrupt: 